## Lab 3
### Regresja Logistyczna
#### Autor: Franciszek

In [ ]:
import csv
import math
import numpy as np
import PIL.Image
import IPython.display
from matplotlib import pyplot as plt
from copy import deepcopy

In [ ]:
def sigmoid(y):
    return 1.0 / (1.0 + np.exp(-y))

In [ ]:
temp = np.linspace(-10, 10, 10000)
temp = np.reshape(temp, [-1, 1])
y = sigmoid(temp)
y = np.reshape(y, [-1, 1])
plt.figure()
plt.plot(temp, y)
plt.show()

In [ ]:
def h_fun(X, theta):
    '''
    :param X: ndarray postaci (n+1, m).
    :param theta: macierz parametrów do optymalizacji postaci (n+1, 1)
    :return: 
    '''
    z = np.matmul(theta.T, X)
    h = sigmoid(z)
    if np.any(h == 0):
        h += 1e-30
    if np.any(h == 1):
        h -= 1e-30
    return h

In [ ]:
def calculate_cost(X, y, theta):
    '''
    :param X: ndarray postaci (n+1, m).
    :param y: ndarray z wartościami referencyjnymi o wymiarze (1, m)
    :param theta: macierz parametrów do optymalizacji postaci (n+1, 1)
    :return: wartość f. kosztu
    '''
    m = X.shape[1]
    h = h_fun(X, theta)
    y_1 = -y*np.log(h)
    y_0 = -(1-y)*np.log(1-h)
    return (y_1 + y_0).sum() / m

In [ ]:
def computeDerivativeMultivariable(X, Y, theta):
    '''
    :param X: ndarray postaci (n+1, m).
    :param y: ndarray z wartościami referencyjnymi o wymiarze (1, m)
    :param theta: macierz parametrów do optymalizacji postaci (n+1, 1)
    :return: wartość f. kosztu
    '''
    n,m = X.shape # number of features, number of examples
    
    diff = (h_fun(X, theta) - Y)
    gradient = np.matmul(diff, X.T)
    gradient = gradient / m

    return gradient.T

In [ ]:
def LogisticRegression(X, Y, theta_in, learningRate, epsilon):
    i = 0
    costList = []
    derivatveList = []
    thetaList = []

    theta = deepcopy(theta_in)

    while(i < 2 or abs((costList[i-1] - costList[i-2])) > epsilon):
        derivative = computeDerivativeMultivariable(X, Y, theta)
        derivatveList.append(derivative)

        theta = theta - learningRate * derivative
        thetaList.append(theta)

        cost = calculate_cost(X, Y, theta)
        costList.append(cost)
        
        print(f"Iteration: {i}, Cost: {costList[-1]}")

        i += 1

    return [theta, i, costList]

In [ ]:
def computeResult(theta, example, X_scale, y_scale):
    return sigmoid(np.matmul(theta.T, example/X_scale) * y_scale)

In [ ]:
X = np.array([[1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0],
              [0.3, 0.8, 1.7, 2.4, 2.9, 3.1, 4.5, 6.1]])
y = np.array([[0.0, 0.0, 0.0, 0.0, 1.0, 1.0, 1.0, 1.0]])

theta_in = np.zeros((2, 1))
eps = 1e-5  # akceptowalna różnica dla kolejnych wartości funkcji kosztu 
alpha = 0.001  # learning rate
theta_0 = 0  # - wartości początkowe parametrów modelu
theta_1 = 0

In [ ]:
theta, i, costList = LogisticRegression(X, y, theta_in, alpha, eps)

In [ ]:
data = np.linspace(0, 7, 1000)
temp = np.vstack([np.ones([1, 1000]), data])
data = np.reshape(data, [-1, 1])
prediction = computeResult(theta, temp, 1, 1)
prediction = np.reshape(prediction, [-1, 1])

In [ ]:
X = np.array([[1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0],
              [0.3, 0.8, 1.7, 2.4, 2.9, 3.1, 4.5, 6.1]])
results = computeResult(theta, X, 1, 1)
results = np.where(results <= 0.5, 0, 1)

x = np.reshape(X[1, :], [1, -1])
results = np.reshape(results, [1, -1])

In [ ]:
plt.figure()
plt.plot(data, prediction, 'g')
plt.scatter(x, results)
plt.show()

### zadanie 2.

In [ ]:
with open("./admission.txt") as f:
    csv_reader = csv.reader(f)
    examples = []
    for row in csv_reader:
        examples.append([float(cell) for cell in row])
    data = np.array(examples)
    y = data[:, 2:3].T
    x = data[:, :2].T
    X = np.concatenate([np.ones([1, x.shape[1]]), x])
 
print(f'{X.shape=}')
print(f'{y.shape=}')

In [ ]:
value0 = X[:, (y == 0)[0]]
value1 = X[:, (y == 1)[0]]

plt.figure()
plt.scatter(value0[1, :], value0[2, :], marker='o')
plt.scatter(value1[1, :], value1[2, :], marker='x')

In [ ]:
theta_in = np.zeros((3, 1))
eps = 1e-7  # akceptowalna różnica dla kolejnych wartości funkcji kosztu 
alpha = 0.001  # learning rate

In [ ]:
theta, i, costList = LogisticRegression(X, y, theta_in, alpha, eps)

In [ ]:
x1_db = np.arange(X[1, :].min(), X[1, :].max(), 1)
x2_db = -theta[0, 0]/theta[2, 0] - theta[1, 0]/theta[2, 0]*x1_db
plt.figure()
plt.plot(x1_db, x2_db, '-m')
plt.scatter(value0[1, :], value0[2, :], marker='o')
plt.scatter(value1[1, :], value1[2, :], marker='x')
plt.show()

### Zadanie 3.

In [ ]:
with open("./admission.txt") as f:
    csv_reader = csv.reader(f)
    examples = []
    for row in csv_reader:
        examples.append([float(cell) for cell in row])
    data = np.array(examples)
    y = data[:, 2:3].T
    x1 = data[:, :1].T
    x2 = data[:, 1:2].T
    X = np.concatenate([np.ones([1, x1.shape[1]]), x1, x2, x1**2])

In [ ]:
X_max = X.max(axis=1)
X_max.shape = [X_max.shape[0], 1]
X_min = X.min(axis=1)
X_min[0] = 0
X_min.shape = [X_min.shape[0], 1]
X_normalized = (X - X_min) / (X_max - X_min)

In [ ]:
value0 = X[:, np.where(y == 0)[1]]
value1 = X[:, np.where(y == 1)[1]]

plt.figure()
plt.scatter(value0[1, :], value0[2, :], marker='o')
plt.scatter(value1[1, :], value1[2, :], marker='x')
plt.ylabel('x2')
plt.xlabel('x1')
plt.show()

In [ ]:
theta_in = np.ones((4, 1))
eps = 1e-7 # akceptowalna różnica dla kolejnych wartości funkcji kosztu 
alpha = 0.001  # learning rate

In [ ]:
theta, i, costList = LogisticRegression(X_normalized, y, theta_in, alpha, eps)

In [ ]:
x1_db = np.arange(X[1, :].min(), X[1, :].max(), 1)
x1_db_normalized = (x1_db - X_min[1, :]) / (X_max[1, :] - X_min[1, :])
x2_db = -theta[0, 0]/theta[2, 0] - theta[1, 0]/theta[2, 0]*x1_db_normalized - theta[3, 0] / theta[2, 0] * (np.power(x1_db_normalized, 2))
x2_db = x2_db * (X_max[2, :] - X_min[2, :]) + X_min[2, :]
plt.figure()
plt.plot(np.reshape(x1_db, [-1, 1]), np.reshape(x2_db, [-1, 1]))
plt.scatter(value0[1, :], value0[2, :], marker='o')
plt.scatter(value1[1, :], value1[2, :], marker='x')
plt.ylabel('x2')
plt.xlabel('x1')
plt.show()

In [ ]:
print(computeResult(theta, [[1],[60], [20], [3600]] - X_min, X_max - X_min, 1))
print(computeResult(theta, [[1], [80], [80], [80**2]] - X_min, X_max -X_min, 1))

### Zadanie 4.

In [ ]:
def computeResult(theta, example, X_scale, y_scale):
    return np.where(sigmoid(np.matmul(theta.T, example/X_scale) * y_scale) <= 0.5, 0, 1)

In [ ]:
from sklearn import datasets
data = datasets.load_digits()
y = data['target']
x = data['data']
plt.imshow(np.reshape(x[0, :] * 255, [8, 8]))

In [ ]:
from sklearn.model_selection import train_test_split
 
x, x_test, y, y_test = train_test_split(x, y, random_state=1234)

In [ ]:
m,n = x.shape # number of examples, number of features
m_test, n_test = x_test.shape
x_maximum = np.max(x)
x_scaled = x / x_maximum
x_test_temp = np.concatenate([np.ones((1, m_test)), x_test.T])

theta_list = []
cost_list = []
eps = 1e-5
alpha = 1e-1

In [ ]:
for class_ in range(10):

    y_temp = (y == class_).astype(np.int8)
    x_temp = np.concatenate([np.ones((1, m)), x_scaled.T])
    theta_in = np.zeros((n + 1, 1))

    theta, i, costList = LogisticRegression(x_temp, y_temp, theta_in, alpha, eps)

    theta_list.append(theta)
    cost_list.append(costList[-1])

In [ ]:
precision_list = []
recall_list = []

In [ ]:
for class_ in range(10):

    result = computeResult(theta_list[class_], x_test_temp, x_maximum, 1)
    y_test_temp = (y_test == class_).astype(np.int8)

    true_positive = np.logical_and(result == 1, y_test_temp == 1).sum()
    false_positive = np.logical_and(result == 1, y_test_temp == 0).sum()
    false_negative = np.logical_and(result == 0, y_test_temp == 1).sum()

    precision = true_positive / (true_positive + false_positive)
    recall = true_positive / (true_positive + false_negative)
    
    precision_list.append(precision)
    recall_list.append(recall)

In [ ]:
print("class|precision|recall")
print("-"*25)

for class_ in range(10):
    print(f'{class_:>5}|{precision_list[class_]:>9.6f}|{recall_list[class_]:>9.6f}')

In [ ]:
result = computeResult(theta_list[6], x_test_temp, x_maximum, 1)
indexes = np.where(result == 1)
print(indexes[1])
plt.imshow(np.reshape(x_test[37, :], [8, 8]))
plt.show()
plt.imshow(np.reshape(x_test[39, :], [8, 8]))
plt.show()